# The process of analyzing the spatial transcriptome data of mouse diaphragm

### convert bin1 to bin20

In [1]:
import os
import sys

import stDrosophila as sd
folder = "/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_raw/lasso_bin1"
files = [os.path.join(root, filename)for root, dirs, files in os.walk(folder)for filename in files]
files.sort()
for file in files:
    bin1_data = sd.io.read_lasso(filename=file)
    file_name = file.split("/")[-1]

    print(f"----{file_name} start!")
    data = sd.pp.bin1tobinx(bin1_data, 20,
                            save=f"/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_raw/lasso_bin20/{file_name}", save_format="gzip")
    del data
    print(f"----{file_name} successfully!")

----mouse_diaphragm_C1.gem.gz start!
----mouse_diaphragm_C1.gem.gz successfully!
----mouse_diaphragm_C2.gem.gz start!
----mouse_diaphragm_C2.gem.gz successfully!
----mouse_diaphragm_C3.gem.gz start!
----mouse_diaphragm_C3.gem.gz successfully!


## cropping lasso file based on ssDNA image (semi-automatic)
   1. Preprocess the ssDNA image and lasso image through `sd.pp.pre_photo` and `sd.pp.pre_lasso` respectively;
   2. Register the image and cut out the required image position through photoshop;
   3. Process the lasso matrix with the photoshopped image through `sd.pp.cropbyphoto`.

ssDNA preprocessing

In [ ]:
import cv2
import stDrosophila as sd

tif = cv2.imread("/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/ssDNA_image/FP200000514BR_C1.tif", 2)
pre_tif = sd.pp.pre_photo(img=tif, rectangle_crop=None, coords_flip=None,
                          ehtranfer={"method": "local", "cliplimit": 15}, show=True, save="mouse_diaphragm_C1_tif.tif")

lasso preprocessing

In [ ]:
import pandas as pd
import stDrosophila as sd

lasso = pd.read_csv("/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_raw/lasso_bin20/mouse_diaphragm_C1.gem.gz", sep="\t", compression="gzip")
pre_lasso, pre_lasso_img = sd.pp.pre_lasso(data=lasso, rectangle_crop=None, ehtranfer={"method": "local", "cliplimit": 10}, gray_factor=None,
                                           show=True, save_img="/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_pre/mouse_diaphragm_C1_lasso.tif",
                                           save_lasso="/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_pre/mouse_diaphragm_C1_pre.txt")

cropping after photoshop

In [ ]:
import cv2
import pandas as pd
import stDrosophila as sd

tif = cv2.imread("/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_pre/mouse_diaphragm_C1/img/mouse_diaphragm_C1_ps.tif", 2)
lasso = pd.read_csv(r"/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_pre/mouse_diaphragm_C1/mouse_diaphragm_C1_pre.txt", sep="\t")
new_lasso = sd.pp.cropbyphoto(data=lasso, img=tif, background=255, save=r"/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_pre/mouse_diaphragm_C1.txt.gz")
print(new_lasso)

visualize cropped lasso

In [2]:

import stDrosophila as sd
data = sd.io.read_lasso(filename=r"/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_pre/mouse_diaphragm_C3.txt")
adata = sd.io.lasso2adata(data=data, slice="mouse_diaphragm_C3", binsize=20)
sd.pl.spatial_plot(adata=adata, cluster_col=None, save="/media/yao/Elements SE/BGI_Paper/mouse_diaphragm/lasso_pre/mouse_diaphragm_C3_coords.png",
                   slice_col="slice", spot_size=1)

... storing 'slice' as categorical


<Figure size 432x288 with 0 Axes>

## construct an AnnData object suitable for downstream analysis